# Proyecto de aprendizaje automático de extremo a extremo

### Trabajar con datos reales

Si queremos obtener resultados confiables, es fundamental trabajar con datos reales y no con datos inventados. Los modelos de aprendizaje automático solo pueden aprender patrones útiles si la información que reciben representa el mundo tal como es, con sus imperfecciones, ruido y complejidad.

Afortunadamente, existen múltiples plataformas que recopilan y publican conjuntos de datos abiertos. Muchas de ellas cuentan con comunidades activas que actualizan y comparten información de forma continua, lo que permite acceder a datos variados y de buena calidad.

Algunos de los repositorios de datos abiertos más populares son:

* **OpenML.org**
* **Kaggle.com**
* **PapersWithCode.com**

En este capítulo trabajaremos con el conjunto de datos **California Housing Prices**, que contiene información de un censo realizado en 1990. Nuestro objetivo será utilizar estos datos para construir un modelo capaz de predecir el precio de las viviendas en distintas zonas de California.